In [1]:
# Imports
import re
import spacy
import string
import os
import numpy as np
import pandas as pd
import dateparser
import seaborn as sns
from datetime import datetime, timezone, timedelta
import pickle
from IPython.display import HTML

In [2]:
def html(df):
    display(HTML(df.to_html(render_links=True, escape=False)))

In [3]:
MinFin = pd.read_pickle('data/MinFin/MinFin.pkl.gz', compression='gzip')
min2023 = pd.read_pickle('data/MinFin/min2023besluit.pkl.gz', compression='gzip')
min20231 = min2023[min2023.index.get_level_values(1) == 1]

In [4]:
comb = pd.merge(MinFin, min20231, on='dc_title')

print(len(set(min2023['dc_title']).intersection(set(MinFin['dc_title']))))

20


In [5]:
def is_same_dagtekening(row):
    dtextract = row['dagtekening_complete']
    dtantwoord = row['Besluit']
    return dtextract == dtantwoord

def is_same_verzoek(row):
    dtextract = row['verzoek_datum']
    dtantwoord = row['Ingediend']
    return dtextract == dtantwoord

In [6]:
comb['dagtekening_same'] = comb.apply(lambda x: is_same_dagtekening(x), axis=1)
comb['verzoek_same'] = comb.apply(lambda x: is_same_verzoek(x), axis=1)

In [7]:
def verschil_extractor(row):
    dagtekening = row['dagtekening_complete']
    verzoek = row['verzoek_datum']
    if isinstance(dagtekening, datetime) and isinstance(verzoek, datetime):
        return dagtekening - verzoek
    else:
        return None
    
def verschil_excel(row):
    dagtekening = row['Besluit']
    verzoek = row['Ingediend']
    if isinstance(dagtekening, datetime) and isinstance(verzoek, datetime):
        return dagtekening - verzoek
    else:
        return None
    
def td_to_days(td):
    if td != None:
        return td.days
    
comb['verschil_extractor'] = None
comb['verschil_extractor'] = comb.apply(lambda x: td_to_days(verschil_extractor(x)), axis=1)

comb['verschil_excel'] = None
comb['verschil_excel'] = comb.apply(lambda x: td_to_days(verschil_excel(x)), axis=1)

## Dropna all

dagtekening_same en verzoek_same: vertellen of de handmatige en mijn datums overeenkomen. <br>
verschil_extractor en verschil_excel: vertellen hoeveel verschil er zit tussen de handmatige en mijn datums <br>
verschil_verschil: vertelt hoeveel dagen ik naast het handmatige verschil zit

In [24]:
comb['verschil_verschil_abs'] = comb['verschil_extractor'] - comb['verschil_excel']
comb['verschil_verschil_perc'] = round(comb['verschil_extractor'] * 100 / comb['verschil_excel'] - 100, 2)
comb[['dagtekening_complete', 'Besluit', 'dagtekening_same', 'verzoek_datum', 'Ingediend', 'verzoek_same', 'verschil_extractor', 'verschil_excel', 'verschil_verschil_perc', 'verschil_verschil_abs']].dropna(how='any')

,dagtekening_complete,Besluit,dagtekening_same,verzoek_datum,Ingediend,verzoek_same,verschil_extractor,verschil_excel,verschil_verschil_perc,verschil_verschil_abs
0,2023-06-15 00:00:00,2023-06-15,True,2023-02-14,2023-02-17,False,121.0,118.0,2.54,3.0
1,2023-06-15 00:00:00,2023-06-15,True,2023-02-14,2023-02-17,False,121.0,118.0,2.54,3.0
2,2023-06-15 00:00:00,2023-06-15,True,2023-02-14,2023-02-17,False,121.0,118.0,2.54,3.0
3,2023-06-15 00:00:00,2023-06-15,True,2023-02-14,2023-02-17,False,121.0,118.0,2.54,3.0
5,2023-07-07 00:00:00,2023-07-07,True,2023-04-23,2023-04-23,True,75.0,75.0,0.00,0.0
6,2023-07-06 00:00:00,2023-07-06,True,2023-01-31,2023-01-31,True,156.0,156.0,0.00,0.0
7,2023-02-01 00:00:00,2023-02-01,True,2022-09-11,2022-09-12,False,143.0,142.0,0.70,1.0
8,2023-07-13 00:00:00,2023-07-13,True,2021-05-03,2021-05-20,False,801.0,784.0,2.17,17.0
9,2023-07-13 00:00:00,2023-07-13,True,2023-01-09,2023-01-09,True,185.0,185.0,0.00,0.0
10,2023-07-13 00:00:00,2023-07-13,True,2023-03-23,2023-03-23,True,112.0,112.0,0.00,0.0


In [26]:
comb[['verschil_extractor', 'verschil_excel']].describe()

,verschil_extractor,verschil_excel
count,21.000000,24.000000
mean,276.380952,165.791667
std,349.396262,156.031485
min,28.000000,28.000000
25%,121.000000,98.000000
50%,131.000000,118.000000
75%,268.000000,163.250000
max,1535.000000,784.000000


## Alles

In [18]:
comb[['dagtekening_complete', 'Besluit', 'dagtekening_same', 'verzoek_datum', 'Ingediend', 'verzoek_same', 'verschil_extractor', 'verschil_excel', 'verschil_verschil_perc', 'verschil_verschil_abs']]

,dagtekening_complete,Besluit,dagtekening_same,verzoek_datum,Ingediend,verzoek_same,verschil_extractor,verschil_excel,verschil_verschil_perc,verschil_verschil_abs
0,2023-06-15 00:00:00,2023-06-15,True,2023-02-14,2023-02-17,False,121.0,118.0,2.54,3.0
1,2023-06-15 00:00:00,2023-06-15,True,2023-02-14,2023-02-17,False,121.0,118.0,2.54,3.0
2,2023-06-15 00:00:00,2023-06-15,True,2023-02-14,2023-02-17,False,121.0,118.0,2.54,3.0
3,2023-06-15 00:00:00,2023-06-15,True,2023-02-14,2023-02-17,False,121.0,118.0,2.54,3.0
4,None,2023-06-28,False,2023-05-11,2023-05-11,True,NaN,48.0,NaN,NaN
5,2023-07-07 00:00:00,2023-07-07,True,2023-04-23,2023-04-23,True,75.0,75.0,0.00,0.0
6,2023-07-06 00:00:00,2023-07-06,True,2023-01-31,2023-01-31,True,156.0,156.0,0.00,0.0
7,2023-02-01 00:00:00,2023-02-01,True,2022-09-11,2022-09-12,False,143.0,142.0,0.70,1.0
8,2023-07-13 00:00:00,2023-07-13,True,2021-05-03,2021-05-20,False,801.0,784.0,2.17,17.0
9,2023-07-13 00:00:00,2023-07-13,True,2023-01-09,2023-01-09,True,185.0,185.0,0.00,0.0
